# Imports

In [ ]:
import pandas as pd
from hbt_tools import utils_hbt as uh

# Ablauf

In [ ]:
id = 466
date_format = '%d.%m.%Y %H:%M'

periods = [
    ("28.09.2021 14:22", "01.12.2021 08:49"),
    ("01.12.2021 08:49", "08.12.2021 11:04"),
    ("08.12.2021 11:04", "29.12.2021 18:13"),
    ("29.12.2021 18:13", "04.01.2022 19:56"),
    ("07.02.2022 10:06", "01.04.2022 13:42"),
    ("01.04.2022 13:42", "26.04.2022 16:58"),
    ("26.04.2022 16:58", "23.05.2022 22:12"),
    ("23.05.2022 22:12", "01.06.2022 13:34"),
    ("02.06.2022 05:33", "07.06.2022 23:22"),
    ("07.06.2022 23:22", "22.06.2022 23:09"),
    ("22.06.2022 23:09", "24.06.2022 16:48"),
    ("24.06.2022 16:48", "04.07.2022 09:01"),
    ("04.07.2022 09:01", "04.07.2022 23:05"),
    ("04.07.2022 23:05", "23.07.2022 02:53"),
    ("23.07.2022 02:53", "15.08.2022 19:26"),
    ("15.08.2022 19:26", "19.08.2022 07:28"),
    ("22.09.2022 10:29", "28.09.2022 19:49"),
    ("28.09.2022 19:49", "08.10.2022 20:49"),
    ("08.10.2022 20:49", "04.11.2022 02:18"),
    ("04.11.2022 12:33", "05.11.2022 12:13"),
    ("05.11.2022 12:13", "10.12.2022 06:44"),
    ("10.12.2022 06:44", "24.12.2022 03:13"),
    ("24.12.2022 03:13", "26.12.2022 19:20"),
    ("26.01.2023 13:39", "12.03.2023 03:31"),
    ("12.03.2023 03:31", "25.03.2023 01:04"),
    ("25.03.2023 01:04", "03.04.2023 14:47"),
    ("03.04.2023 15:04", "24.04.2023 05:35"),
    ("24.04.2023 05:35", "01.05.2023 21:06"),
    ("01.05.2023 21:06", "10.05.2023 03:28")

]

date_from = periods[0][0]
date_to = periods[-1][-1]

q_ab = uh.hbtdb_get_data(id, date_from, date_to, date_format)
q_ab.index = pd.to_datetime(q_ab.index, utc=True)

In [ ]:
q_ab['delta_t'] = q_ab.index
delta_t = q_ab['delta_t'] - q_ab['delta_t'].shift()
q_ab['delta_t'] = delta_t
q_ab['volumen'] = q_ab['wert'] * q_ab['delta_t'].dt.total_seconds()

In [ ]:
df_abfluss = pd.DataFrame(columns=['von', 'bis', 'volumen'])
for i, p in enumerate(periods):
    date_start = pd.to_datetime(p[0], format=date_format)
    date_start = date_start.tz_localize('Europe/Zurich')
    date_end = pd.to_datetime(p[1], format=date_format)
    date_end = date_end.tz_localize('Europe/Zurich')

    q_ab_period = q_ab[(q_ab.index>=date_start) & (q_ab.index<=date_end)]
    df_temp = pd.DataFrame({'von': [p[0]], 'bis': [p[1]],
        'volumen': [q_ab_period['volumen'].sum() / 1e3]})
    
    df_abfluss = pd.concat((df_abfluss, df_temp))

In [ ]:
df_abfluss